# Night Log

v 20240912.1

In [ ]:
# Parameters. Set defaults here.
# Times Square replaces this cell with the user's parameters.
record_limit = '10'
day_obs = '2024-09-04' # 'TODAY'  # TODAY, YESTERDAY, YYYY-MM-DD
number_of_days = '1'  # Total number of days of data to display (ending on day_obs)

In [ ]:
# Only use packages available in the Rubin Science Platform
import requests
from collections import defaultdict
import pandas as pd
from pprint import pp
from urllib.parse import urlencode
from IPython.display import display, Markdown, display_markdown
from matplotlib import pyplot as plt
import os
from datetime import datetime, date, timedelta
#! from rubin_scheduler.site_models import Almanac

In [ ]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = int(record_limit)

match day_obs.lower():
    case 'today':
        date = datetime.now().date()
    case 'yesterday':
        date = datetime.now().date()-timedelta(days=1)
    case _:
        date = datetime.strptime(day_obs, '%Y-%m-%d').date()
# date:  is EXLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
min_day_obs = (date - timedelta(days=days-1)).strftime('%Y%m%d') # Inclusive
max_day_obs = (date + timedelta(days=1)).strftime('%Y%m%d') # prep for Exclusive

response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

In [ ]:
# TODO Set default env to "usdf" and try before PUSH to repo.
summit = 'https://summit-lsp.lsst.codes'
usdf = 'https://usdf-rsp-dev.slac.stanford.edu'
tucson = 'https://tucson-teststand.lsst.codes'

# Use server=tucson for dev testing.
# Use server=usdf for push to develop.   TODO
server = os.environ.get('EXTERNAL_INSTANCE_URL', usdf)

In [ ]:
print(f'Report from {server} over {number_of_days} nights'
      f' from {min_day_obs} to {date}. ')

In [ ]:
# For Times Square, comment out next line and past next cell with contents of local python file.
from lsst.ts.logging_and_reporting.source_adapters import ExposurelogAdapter, NarrativelogAdapter, keep_fields
# Once our logrep package has been installed in RSP, we can use the simpler "import"

In [ ]:
# TODO
# Comment out the import in the cell above this one.
# Paste contents of source_adapters.py in new cell below this one.

# Exposure Log

In [ ]:
exposure_adapter = ExposurelogAdapter(server_url=server)
exposure_url = exposure_adapter.source_url
try:
    exposure_recs = exposure_adapter.get_messages(limit=limit,
                                        min_day_obs=min_day_obs,
                                        max_day_obs=max_day_obs,
                                       )
except Exception as err:
    exposure_recs = []
    msg = f'ERROR getting records from {exposure_url=}: {err=}'
    raise Exception(msg)

In [ ]:
print(f'Retrieved {len(exposure_recs)} records from {exposure_url}')

In [ ]:
if exposure_recs:
    keep_fields(['message_text','date_added'], exposure_recs)
    new_column_names = dict(message_text='message', date_added='date')
    df = pd.DataFrame(exposure_recs).rename(columns=new_column_names)

    display(Markdown(f'## Exposure log '
                     f'for {number_of_days} day {min_day_obs} to {max_day_obs}'
                     ))
    for index,row in df.iterrows():
        print(f"{datetime.fromisoformat(df.iloc[0]['date']).date()}: "
              f"{row['message']}"
        )

In [ ]:
display(Markdown(f"### <font color='red'>{exposure_url}/exposures/ Not yet functional on USDF</font>"))
gaps = exposure_adapter.get_observation_gaps()
if gaps:
    for instrument, day_gaps in gaps.items():
        display(Markdown(f'### Date vs Observation Gap (minutes) for {instrument=!s}'))
        x,y = zip(*day_gaps.items())
        df = pd.DataFrame(dict(day=x,minutes=y))
        df.plot.bar(x='day', y='minutes', title=f'{instrument=!s}')
else:
    print(f'No Observation Gaps found in exposures.')

# Narrative Log


In [ ]:
narrative_adapter = NarrativelogAdapter(server_url=server)
narrative_url = narrative_adapter.source_url
try:
    # date like '2000-01-02 12:00:00'
    # str(datetime(2000, 1, 2, 12, 0, 0))
    min_date = str(datetime.strptime(min_day_obs,'%Y%m%d'))
    max_date = str(datetime.strptime(max_day_obs,'%Y%m%d'))
    print(f'Get data from {narrative_url}: {min_date} to {max_date}')
    narrative_recs = narrative_adapter.get_messages(
        limit=limit,
        min_date_end=min_date,
        max_date_end=max_date
    )
except Exception as err:
    narrative_recs = []
    msg = f'ERROR getting records from {narrative_url}: {err=}'
    raise Exception(msg)

print(f'Retrieved {len(narrative_recs)} records.')

In [ ]:
if narrative_recs:
    keep_fields(['message_text','date_added'], narrative_recs)
    new_column_names = dict(message_text='message', date_added='date')
    df = pd.DataFrame(narrative_recs).rename(columns=new_column_names)
    user_df = df  # [['date','message']]

    display(Markdown(f'## Narrative log'))
    for index,row in user_df.iterrows():
        print(f"{datetime.fromisoformat(user_df.iloc[0]['date']).date()}: "
              f"{row['message']}"
        )